## Training dataset with tiling

Dataset with smaller tiles -> 12 4x4 grid

### setting up

In [1]:
# info about current GPU
!nvidia-smi

Mon Oct 28 14:52:21 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.03                 Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070      WDDM  |   00000000:05:00.0  On |                  N/A |
| 52%   49C    P0             63W /  270W |    1228MiB /   8192MiB |     12%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Setting up the environment & base path of project
import os
HOME = os.getcwd()
print(HOME)

# Installing ultralytics
%pip install ultralytics

from IPython.display import display, Image, clear_output
clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO


Ultralytics 8.3.23  Python-3.11.5 torch-2.6.0.dev20241026+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
Setup complete  (16 CPUs, 31.9 GB RAM, 839.5/930.9 GB disk)


In [3]:
# installing roboflow, this is where our dataset is located
%pip install roboflow

# importing the .env file for the api key
from dotenv import load_dotenv
load_dotenv("../../.env")
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")
print(ROBOFLOW_API_KEY)
# downloading the dataset
from roboflow import Roboflow
rf = Roboflow(ROBOFLOW_API_KEY)
project = rf.workspace("nathanneve").project("nematode-v1")
version = project.version(7)
dataset = version.download("yolov11")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
None
loading Roboflow workspace...
loading Roboflow project...


#### installation of pytorch, make sure the version of pytorch corresponds to the locally installed version of cuda
in this case cuda version needs to be 12.4

In [4]:
# pytorch is nessesary for gpu acceleration with jupyter notebooks
# https://pytorch.org/get-started/locally/
# nvcc --version -> checks installed version of CUDA


# https://stackoverflow.com/questions/75103127/getting-notimplementederror-could-not-run-torchvisionnms-with-arguments-fr
# %pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu124

import torch
print("CUDA Available: ", torch.cuda.is_available())  # Expect True
if torch.cuda.is_available():
    print("Current CUDA Device Index: ", torch.cuda.current_device())
    print("CUDA Device Name: ", torch.cuda.get_device_name(0))
else:
    print("CUDA device not accessible.")

import torchvision
print("torchvision CUDA: ", torchvision.ops.nms is not None)



CUDA Available:  True
Current CUDA Device Index:  0
CUDA Device Name:  NVIDIA GeForce RTX 3070
torchvision CUDA:  True


### Training

In [5]:
# command if training on google collab
# yolo task=detect mode=train model=yolov11n.pt data={dataset.location}/data.yaml epochs=25 imgsz=640 plots=True
from ultralytics import YOLO

my_data = f"{dataset.location}/data.yaml"
model = YOLO("yolo11n.pt")
results = model.train(data=my_data, epochs=80, imgsz=640, model="yolov11n.pt")
# reference https://docs.ultralytics.com/modes/train/

100%|██████████| 5.35M/5.35M [00:02<00:00, 1.98MB/s]


Ultralytics 8.3.23  Python-3.11.5 torch-2.6.0.dev20241026+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolov11n.pt, data=c:\_repos_\2425-FW-001-Aaltjes-tellen\code\Notebooks Tiling\Nematode-v1-7/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fal

train: Scanning C:\_repos_\2425-FW-001-Aaltjes-tellen\code\Notebooks Tiling\Nematode-v1-7\train\labels... 10320 images, 3736 backgrounds, 0 corrupt: 100%|██████████| 10320/10320 [00:11<00:00, 868.77it/s] 


train: New cache created: C:\_repos_\2425-FW-001-Aaltjes-tellen\code\Notebooks Tiling\Nematode-v1-7\train\labels.cache


val: Scanning C:\_repos_\2425-FW-001-Aaltjes-tellen\code\Notebooks Tiling\Nematode-v1-7\valid\labels... 704 images, 265 backgrounds, 0 corrupt: 100%|██████████| 704/704 [00:01<00:00, 558.78it/s]

...

      80/80      2.36G      1.059     0.6128      1.064         15        640: 100%|██████████| 645/645 [01:39<00:00,  6.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:04<00:00,  4.70it/s]

                   all        704        834      0.814      0.859       0.87      0.667



80 epochs completed in 2.160 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 5.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.23  Python-3.11.5 torch-2.6.0.dev20241026+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:04<00:00,  4.58it/s]


                   all        704        834      0.814      0.857      0.871      0.667
                   egg        359        625      0.927      0.919      0.937      0.722
                 shell        163        209      0.701      0.794      0.806      0.613
Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\train


### Validation

In [6]:
%cd {HOME}
# command if training on google collab
# yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml
from ultralytics import YOLO

'''
⚠ NOTE: selections do not prevent you from specifying a combination for a model that doesn't exist.
Reference the documentation for valid model specifications: https://docs.ultralytics.com/models
'''
my_data = f"{dataset.location}/data.yaml"
model_path = f"{HOME}/runs/detect/train/weights/best.pt"
model = YOLO(model_path)
results = model.val(data=my_data)
# reference https://docs.ultralytics.com/modes/val/

c:\_repos_\2425-FW-001-Aaltjes-tellen\code\Notebooks Tiling
Ultralytics 8.3.23  Python-3.11.5 torch-2.6.0.dev20241026+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\_repos_\2425-FW-001-Aaltjes-tellen\code\Notebooks Tiling\Nematode-v1-7\valid\labels.cache... 704 images, 265 backgrounds, 0 corrupt: 100%|██████████| 704/704 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:05<00:00,  7.48it/s]


                   all        704        834      0.816      0.857      0.872      0.668
                   egg        359        625      0.927      0.919      0.937      0.724
                 shell        163        209      0.704      0.794      0.806      0.612
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\val


### Testing

In [ ]:
%cd {HOME}
# command if training on google collab
# yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

src = f"{dataset.location}/test/images"
print("Model names:", getattr(model, 'names', "No names attribute found"))
from ultralytics import YOLO

model = YOLO(model_path)
results = model.predict(source=src, conf=0.25)
# save=True parameter saves the results to runs/detect/predict
# reference https://docs.ultralytics.com/models/rtdetr/